# Read in data

In [8]:
import pandas as pd
import numpy as np

In [9]:
# Read in data
df = pd.read_csv('../Data/preprocessedData.csv')
df.head()

,Age,Height,Weight,Veg_Consump,Water_Consump,Meal_Count,Phys_Act,Time_E_Dev,Body_Level,Gender_Female,...,Food_Between_Meals_no,Fam_Hist_no,Fam_Hist_yes,H_Cal_Burn_no,H_Cal_Burn_yes,Transport_Automobile,Transport_Bike,Transport_Motorbike,Transport_Public_Transportation,Transport_Walking
0,22.547298,1.722461,51.881263,2.663421,1.041110,3.000000,0.794402,1.391948,Body Level 1,1,...,0,0,1,1,0,0,0,0,1,0
1,19.799054,1.743702,54.927529,2.000000,2.847264,3.289260,1.680844,2.000000,Body Level 1,0,...,0,0,1,1,0,0,0,0,1,0
2,17.823438,1.708406,50.000000,1.642241,1.099231,3.452590,0.418875,1.000000,Body Level 1,1,...,0,1,0,1,0,0,0,0,1,0
3,19.007177,1.690727,49.895716,1.212908,1.029703,3.207071,2.000000,1.000000,Body Level 1,1,...,0,1,0,1,0,0,0,0,1,0
4,19.729250,1.793315,58.195150,2.508835,2.076933,3.435905,2.026668,1.443328,Body Level 1,0,...,0,0,1,1,0,1,0,0,0,0


## logisitic regression

In [10]:
#  see the accuracy before balancing the data
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# the target is Body_Level
X = df.drop('Body_Level', axis=1)
y = df['Body_Level']

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create an LogisticRegression classifier
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

# Make predictions on the test set
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(accuracy_score(y_test, y_pred)))

# classification report
print(classification_report(y_test, y_pred))


# Print the confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

Accuracy of logistic regression classifier on test set: 0.83
              precision    recall  f1-score   support

Body Level 1       0.80      0.90      0.85        31
Body Level 2       0.76      0.56      0.64        45
Body Level 3       0.73      0.80      0.76        82
Body Level 4       0.92      0.91      0.92       138

    accuracy                           0.83       296
   macro avg       0.80      0.79      0.79       296
weighted avg       0.83      0.83      0.82       296

Confusion Matrix:
[[ 28   3   0   0]
 [  7  25  13   0]
 [  0   5  66  11]
 [  0   0  12 126]]


c:\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## SVM

In [11]:
from sklearn import svm
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# the target is Body_Level
X = df.drop('Body_Level', axis=1)
y = df['Body_Level']

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create an SVM classifier
clf = svm.SVC(kernel='linear')

# Train the SVM classifier
clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# classification report
print(classification_report(y_test, y_pred))

# Print the confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

Accuracy: 0.918918918918919
              precision    recall  f1-score   support

Body Level 1       0.86      1.00      0.93        31
Body Level 2       0.87      0.73      0.80        45
Body Level 3       0.88      0.89      0.88        82
Body Level 4       0.97      0.98      0.97       138

    accuracy                           0.92       296
   macro avg       0.90      0.90      0.90       296
weighted avg       0.92      0.92      0.92       296

Confusion Matrix:
[[ 31   0   0   0]
 [  5  33   7   0]
 [  0   5  73   4]
 [  0   0   3 135]]
